In [21]:
import os,json,csv

txt_file = 'article.txt'
csv_file = 'article.csv'

In [22]:
# txt to csv

rows = []
with open(txt_file,'r',encoding='utf-8') as f:
    rows = f.readlines()
    for i in range(len(rows)):
        try:
            rows[i] = json.loads(rows[i])
        except:
            pass

labels = list(rows[0].keys())

with open(csv_file, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=labels)
    writer.writeheader()
    for row in rows:
        try:
            writer.writerow(row)
        except:
            pass
